In [1]:
from scheduling_functions import *
from scheduling_algorithms import *
from numpy import std
import numpy as np
import sys
import copy
from random import sample, randint, seed, random
from math import isclose, ceil, floor, e, sqrt
from statistics import mean
from decimal import *
from fractions import *
import matplotlib.pyplot as plt
from operator import add
import dill

In [2]:
#creates a bounded random walk:

def random_walk_creation(num_jobs, step_size, random_seed, m, M):
    seed(random_seed)

    ws = [0]*num_jobs
    ws[0] = randint(m,M)
    steps = [randint(-step_size,step_size) for i in range(1,num_jobs)]
    for i in range(1, num_jobs):
        ws[i] = ws[i-1] + steps[i-1]
        ws[i] = min(ws[i], M)
        ws[i] = max(ws[i], m)
    return ws

In [3]:
#creates a job instance given a list of weights and T

def job_instance_creation(ws, D):
    # dictionary: key --> job id
    #            value --> (weight, release time , deadline)
    J = {}
    job_id = 1
    i = 0
    for job_weight in ws:
        J[job_id] = (job_weight , i, i+D)
        i+=1
        job_id+=1
    return J

def job_instance_creation_with_pred(ws, confList, D):
    J = {}
    job_id = 1
    i = 0
    for job_weight in ws:
        J[job_id] = (job_weight , i, i+D, confList[i])
        i+=1
        job_id+=1
    return J

In [4]:
#returns the energy ratio AVR_energy/Optimal_energy

def AVR_energy_ratio(_J, alpha):
    
    J = copy.deepcopy(_J)
    #speed list of average rate
    AVR_speed_list = Avg_rate(J)
    #energy consumption of AVR
    energy_AVR = computeEnergy(AVR_speed_list, alpha)
    
    J = copy.deepcopy(_J)
    #speed list of the optimal schedule
    optimal_alg_speed_list, _ = Optimal_Alg(J)
    #energy consumption of the optimal schedule
    energy_optimal = computeEnergy(optimal_alg_speed_list, alpha)  
    
    return float(energy_AVR)/energy_optimal    

In [5]:
#returns the energy ratio OA_energy/Optimal_energy

def OA_energy_ratio(_J, alpha):
    
    J = copy.deepcopy(_J)
    #speed list of Optimal Available
    OA_speed_list = OptimalOnline(J)
    #energy consumption of Optimal Available
    energy_OA = sum([s**alpha for s in OA_speed_list])
    
    J = copy.deepcopy(_J)
    #speed list of the optimal schedule
    optimal_alg_speed_list, _ = Optimal_Alg(J)
    #energy consumption of the optimal schedule
    energy_optimal = computeEnergy(optimal_alg_speed_list, alpha)    
   
    return float(energy_OA)/energy_optimal

In [6]:
#returns the energy ratio BKP_energy/Optimal_energy

def BKP_energy_ratio(_J, granularity, alpha):
    
    J = copy.deepcopy(_J)
    #energy consumption of the BKP algorithm
    energy_BKP = BKP_alg(J, granularity, alpha)

    J = copy.deepcopy(_J)
    #speed list of the optimal schedule
    optimal_alg_speed_list, _ = Optimal_Alg(J)
    #energy consumption of the optimal schedule
    energy_optimal = computeEnergy(optimal_alg_speed_list, alpha)  
    
    return float(energy_BKP)/energy_optimal    

In [7]:
def LAS_energy_ratio(_J_true, _J_pred, epsilon, alpha, dt):
    
    #compute energy of LAS algorithm
    J_true = copy.deepcopy(_J_true)
    J_pred = copy.deepcopy(_J_pred)
    J_p = {}
    for job_id, job_values in J_pred.items():
        w, r, d, c = job_values
        J_p[job_id] = (w, r, d)
    speed_sol = LAS(J_p, J_true, epsilon, dt, alpha)
    energy_LAS = sum([s**alpha for s in speed_sol])*dt
    
   
    #compute speedlist and energu consumption of the optimal schedule of the true instance
    J_true = copy.deepcopy(_J_true)
    J_pred = copy.deepcopy(_J_pred)
    optimal_alg_speed_list, _ = Optimal_Alg(J_true)
    energy_optimal = computeEnergy(optimal_alg_speed_list, alpha)
    
    return float(energy_LAS)/energy_optimal    

In [8]:
def DCRR_energy_ratio(_J, m, alpha):
    J = copy.deepcopy(_J)
    dcrr_speed_list = DCRR(J, m)
    energy_DCRR = computeEnergy(dcrr_speed_list, alpha)

    J = copy.deepcopy(_J)
    optimal_alg_speed_list, _ = Optimal_Alg(J)
    energy_optimal = computeEnergy(optimal_alg_speed_list, alpha)

    return float(energy_DCRR)/energy_optimal


In [9]:
def yds_m_energy_ratio(_J, m, alpha):
    J = copy.deepcopy(_J)
    yds_speed_list = yds_M(J, m)
    energy_yds_m = computeEnergy(yds_speed_list, alpha)

    J = copy.deepcopy(_J)
    optimal_alg_speed_list, _ = Optimal_Alg(J)
    energy_optimal = computeEnergy(optimal_alg_speed_list, alpha)

    return float(energy_yds_m)/energy_optimal


In [10]:
def SWP_m_energy_ratio(_J_true, _J_pred, m, _epsilon, dt, alpha, confThreshold):
    J_true = copy.deepcopy(_J_true)
    J_pred = copy.deepcopy(_J_pred)
    swp_m_speed_list = swp_m(J_true, J_pred, m, _epsilon, dt, alpha, confThreshold)
    energy_swp_m = sum([s**alpha for s in swp_m_speed_list]) * dt

    J_true = copy.deepcopy(_J_true)
    optimal_alg_speed_list, _ = Optimal_Alg(J_true)
    energy_optimal = computeEnergy(optimal_alg_speed_list, alpha)

    return float(energy_swp_m)/energy_optimal


In [11]:
num_jobs = 40
D = 20

num_of_exp = 20

step_size = 5
M = 80
m = 20

alpha = 3
BKP_granularity = 0.25

dt = 0.01
epsilons = [Fraction(1, 100), Fraction(20, 100), Fraction(40, 100), Fraction(60, 100), Fraction(80, 100)]

In [12]:
J_True_lst = []
w_True_lst = []
for j in range(0, num_of_exp):
    w_true = random_walk_creation(num_jobs, step_size, random_seed=j, M = M, m = m)
    w_True_lst.append(w_true)

    J_True = job_instance_creation(w_true, D)
    J_True_lst.append(J_True)
    

In [13]:
J_pred_lst = []
for j in range(0,num_of_exp):
    w_true = w_True_lst[j]
    
    seed(j)
    error = [randint(-step_size, step_size) for _ in range(0,num_jobs)]
    
    w_pred = list(map(add, w_true, error))
    #job instance creation
    confList = [int(1 / (i + step_size + 1)) % 100 for i in error]
    J_pred = job_instance_creation_with_pred(w_pred, confList, D)
    
    J_pred_lst.append(J_pred)


In [14]:
m_values = [1, 2, 3, 4, 5]
for m in m_values:
    print("m = ", m)
    y_DCRR_scheduling = []
    dummy_y_DCRR_scheduling = []
    for j in range(0,num_of_exp):
        J_true = J_True_lst[j]
        DCRR_scheduling = DCRR_energy_ratio(J_true, m, alpha)
        dummy_y_DCRR_scheduling.append(DCRR_scheduling)

    y_DCRR_scheduling.append(mean(dummy_y_DCRR_scheduling))
    std_DCRR_scheduling = std(dummy_y_DCRR_scheduling)
    
    print("DCRR scheduling: ", y_DCRR_scheduling[-1])
    print("Std of DCRR scheduling ", std_DCRR_scheduling)
    print("=====================")


m =  1
DCRR scheduling:  1.6548588616653448
Std of DCRR scheduling  0.10143523176524126
m =  2
DCRR scheduling:  0.40839974655279987
Std of DCRR scheduling  0.034747084158199965
m =  3
DCRR scheduling:  0.18703757000848545
Std of DCRR scheduling  0.011142550398830239
m =  4
DCRR scheduling:  0.1050903982580648
Std of DCRR scheduling  0.006227994761144435
m =  5
DCRR scheduling:  0.06779028109005993
Std of DCRR scheduling  0.0038981697704482104


In [ ]:
m_values = [1, 2, 3, 4, 5]
for m in m_values:
    print("m = ", m)
    y_yds_m_scheduling = []
    dummy_y_yds_m_scheduling = []
    for j in range(0,num_of_exp):
        J_true = J_True_lst[j]
        yds_m_scheduling = yds_m_energy_ratio(J_true, m, alpha)
        dummy_y_yds_m_scheduling.append(yds_m_scheduling)

    y_yds_m_scheduling.append(mean(dummy_y_yds_m_scheduling))
    std_yds_m_scheduling = std(dummy_y_yds_m_scheduling)
    
    print("YDS_m scheduling: ", y_yds_m_scheduling[-1])
    print("Std of YDS_m scheduling ", std_yds_m_scheduling)
    print("=====================")


m =  1
DCRR scheduling:  1.0
Std of DCRR scheduling  0.0
m =  2
DCRR scheduling:  0.2593990080196109
Std of DCRR scheduling  0.0015130918521132726
m =  3
DCRR scheduling:  0.11933225617025547
Std of DCRR scheduling  0.0008687391989931167
m =  4
DCRR scheduling:  0.06947689782904078
Std of DCRR scheduling  0.0007370390983674397
m =  5
DCRR scheduling:  0.045851587146940764
Std of DCRR scheduling  0.0006495744322636338


In [17]:
for epsilon in epsilons:
    print("EPSILON = ", epsilon)
    y_SWP_m_scheduling = []
    y_LAS_scheduling = []
    
    dummy_y_SWP_m_scheduling = []
    dummy_y_LAS_scheduling = []
    for j in range(0,num_of_exp):
        J_true = J_True_lst[j]
        w_true = w_True_lst[j]
        J_pred = J_pred_lst[j]           
        
        SWP_m_scheduling = SWP_m_energy_ratio(J_true, J_pred, m, epsilon, dt, alpha, confThreshold=0.5)
        LAS_scheduling = LAS_energy_ratio(J_true, J_pred, epsilon, alpha, dt)

        dummy_y_SWP_m_scheduling.append(SWP_m_scheduling)
        dummy_y_LAS_scheduling.append(LAS_scheduling)
    
    y_SWP_m_scheduling.append(mean(dummy_y_SWP_m_scheduling))
    std_SWP_m_scheduling = std(dummy_y_SWP_m_scheduling)
    
    y_LAS_scheduling.append(mean(dummy_y_LAS_scheduling))
    std_LAS_scheduling = std(dummy_y_LAS_scheduling)
    
    print("SWP_m scheduling: ", y_SWP_m_scheduling[-1])
    print("Std of SWP_m scheduling ", std_SWP_m_scheduling)
    print("LAS scheduling: ", y_LAS_scheduling[-1])
    print("Std of LAS scheduling ", std_LAS_scheduling)
    print("=====================")

EPSILON =  1/100
SWP_m scheduling:  0.4251104151810318
Std of SWP_m scheduling  0.009327874020343229
LAS scheduling:  0.3642426438007129
Std of LAS scheduling  0.0016937214906448316
EPSILON =  1/5
SWP_m scheduling:  0.43770854918403446
Std of SWP_m scheduling  0.009657867282850025
LAS scheduling:  0.3738473363826842
Std of LAS scheduling  0.0015112157923862406
EPSILON =  2/5
SWP_m scheduling:  0.44934461602704395
Std of SWP_m scheduling  0.00996278147352321
LAS scheduling:  0.38276397694319664
Std of LAS scheduling  0.0013878115516940733
EPSILON =  3/5
SWP_m scheduling:  0.45969885549297035
Std of SWP_m scheduling  0.010225429034864146
LAS scheduling:  0.39070130054960606
Std of LAS scheduling  0.0013274343881428848
EPSILON =  4/5
SWP_m scheduling:  0.4690045279693895
Std of SWP_m scheduling  0.010457545341848987
LAS scheduling:  0.3979940701313802
Std of LAS scheduling  0.0013388423275934773


### (2) Random predictor


In [18]:
J_pred_lst = []
for j in range(0,num_of_exp):
    seed(j)
    error = [randint(-step_size, step_size) for _ in range(0,num_jobs)]
    
    w_pred = [randint(m,M) for _ in range(0,num_jobs)]
    confList = [randint(0, 100) for _ in range(0, num_jobs)]
    
    #job instance creation
    J_pred = job_instance_creation_with_pred(w_pred, confList, D)
    
    J_pred_lst.append(J_pred)


In [19]:
for epsilon in epsilons:
    print("EPSILON = ", epsilon)
    y_SWP_m_scheduling = []
    y_LAS_scheduling = []
    
    dummy_y_SWP_m_scheduling = []
    dummy_y_LAS_scheduling = []
    for j in range(0,num_of_exp):
        J_true = J_True_lst[j]
        w_true = w_True_lst[j]
        J_pred = J_pred_lst[j]           
        
        SWP_m_scheduling = SWP_m_energy_ratio(J_true, J_pred, m, epsilon, dt, alpha, confThreshold=0.5)
        LAS_scheduling = LAS_energy_ratio(J_true, J_pred, epsilon, alpha, dt)

        dummy_y_SWP_m_scheduling.append(SWP_m_scheduling)
        dummy_y_LAS_scheduling.append(LAS_scheduling)
    
    y_SWP_m_scheduling.append(mean(dummy_y_SWP_m_scheduling))
    std_SWP_m_scheduling = std(dummy_y_SWP_m_scheduling)
    
    y_LAS_scheduling.append(mean(dummy_y_LAS_scheduling))
    std_LAS_scheduling = std(dummy_y_LAS_scheduling)
    
    print("SWP_m scheduling: ", y_SWP_m_scheduling[-1])
    print("Std of SWP_m scheduling ", std_SWP_m_scheduling)
    print("LAS scheduling: ", y_LAS_scheduling[-1])
    print("Std of LAS scheduling ", std_LAS_scheduling)
    print("=====================")

EPSILON =  1/100
SWP_m scheduling:  0.506555077175542
Std of SWP_m scheduling  0.03634145906663663
LAS scheduling:  0.44687834824715306
Std of LAS scheduling  0.04977351981091456
EPSILON =  1/5
SWP_m scheduling:  0.5220724794463341
Std of SWP_m scheduling  0.03764159426194597
LAS scheduling:  0.4418829126938187
Std of LAS scheduling  0.02993995158512824
EPSILON =  2/5
SWP_m scheduling:  0.5363159633872848
Std of SWP_m scheduling  0.038924689216159
LAS scheduling:  0.44244345462946244
Std of LAS scheduling  0.021874491804492514
EPSILON =  3/5
SWP_m scheduling:  0.5488355212883395
Std of SWP_m scheduling  0.04014759774755265
LAS scheduling:  0.4461097616907534
Std of LAS scheduling  0.021190460515744487
EPSILON =  4/5
SWP_m scheduling:  0.5600616052492587
Std of SWP_m scheduling  0.04137263615484002
LAS scheduling:  0.4516243855279908
Std of LAS scheduling  0.022984738141707997


### Misleading predictor


In [20]:
J_pred_lst = []
for j in range(0,num_of_exp):
    w_true = w_True_lst[j]
    
    w_pred = []
    max_W, min_W = 0, 0
    for i in range(0,num_jobs):
        w_pred.append((M-w_true[i]) + m)
        max_W = max(max_W, (M - w_true[i]) + m)
        min_W = min(min_W, (M - w_true[i]) + m)

    confList = [(w - min_W) /(2 * (max_W - min_W)) for w in w_pred]
    
    #job instance creation
    J_pred = job_instance_creation_with_pred(w_pred, confList, D)
    
    J_pred_lst.append(J_pred)


In [21]:
for epsilon in epsilons:
    print("EPSILON = ", epsilon)
    y_SWP_m_scheduling = []
    y_LAS_scheduling = []
    
    dummy_y_SWP_m_scheduling = []
    dummy_y_LAS_scheduling = []
    for j in range(0,num_of_exp):
        J_true = J_True_lst[j]
        w_true = w_True_lst[j]
        J_pred = J_pred_lst[j]           
        
        SWP_m_scheduling = SWP_m_energy_ratio(J_true, J_pred, m, epsilon, dt, alpha, confThreshold=0.5)
        LAS_scheduling = LAS_energy_ratio(J_true, J_pred, epsilon, alpha, dt)

        dummy_y_SWP_m_scheduling.append(SWP_m_scheduling)
        dummy_y_LAS_scheduling.append(LAS_scheduling)
    
    y_SWP_m_scheduling.append(mean(dummy_y_SWP_m_scheduling))
    std_SWP_m_scheduling = std(dummy_y_SWP_m_scheduling)
    
    y_LAS_scheduling.append(mean(dummy_y_LAS_scheduling))
    std_LAS_scheduling = std(dummy_y_LAS_scheduling)
    
    print("SWP_m scheduling: ", y_SWP_m_scheduling[-1])
    print("Std of SWP_m scheduling ", std_SWP_m_scheduling)
    print("LAS scheduling: ", y_LAS_scheduling[-1])
    print("Std of LAS scheduling ", std_LAS_scheduling)
    print("=====================")

EPSILON =  1/100
SWP_m scheduling:  0.5335017976240002
Std of SWP_m scheduling  0.0693449604135492
LAS scheduling:  0.46478568521017466
Std of LAS scheduling  0.05884853718144109
EPSILON =  1/5
SWP_m scheduling:  0.5501944367478215
Std of SWP_m scheduling  0.07394010143465829
LAS scheduling:  0.47804633078885367
Std of LAS scheduling  0.06310783040578541
EPSILON =  2/5
SWP_m scheduling:  0.5653080337825516
Std of SWP_m scheduling  0.07851726793011626
LAS scheduling:  0.490317367442867
Std of LAS scheduling  0.06773298124215718
EPSILON =  3/5
SWP_m scheduling:  0.578565414696391
Std of SWP_m scheduling  0.0828608795352697
LAS scheduling:  0.5011545167307656
Std of LAS scheduling  0.07226000563482878
EPSILON =  4/5
SWP_m scheduling:  0.5903672874777336
Std of SWP_m scheduling  0.08691031354999033
LAS scheduling:  0.5110979868163783
Std of LAS scheduling  0.07659579037118662
